In [26]:
%reload_ext autoreload
%autoreload 2

%gui qt

import time
from tqdm import tqdm
from pathlib import Path
import numpy as np
import numba as nb
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt

import vrExperiment as vre
import vrFunctions as vrf
import basicFunctions as bf
import fileManagement as fm
import redCellGUI as rgui
import dbManagement as dbm

In [3]:
# Immediate next steps for programming work:
# - ROICaT index alignment tools

In [113]:
mouseName = 'CR_Hippocannula6'
dateString = '2022-09-05'
session = '701'
# vrExperiment registration and preprocessing 
# vrReg = vre.vrExperimentRegistration(mouseName, dateString, session, oasis=False, imaging=True)
# vrReg.processTimeline()
# vrReg.processImaging()
# vrReg.processRedCells()
# vrReg.saveParams()
# print(vrReg.sessionPath())
# del vrReg

In session: CR_Hippocannula6/2022-09-05/701, vrFile['rigInfo'] does not exist. Assuming default settings for B2!
Convert mpciROIs.planeIndex to mpciROIs.stackPosition, with [X,Y,Z(planeIndex)] coordinate for each ROI!
Computing red cell features... (usually takes 10-20 seconds)
C:\Users\andrew\Documents\localData\CR_Hippocannula6\2022-09-05\701


In [30]:
mouseName = 'ATL020'
dateString = '2023-05-04'
session = '702'

In [3]:
# vrExperiment registration and preprocessing 
vrReg = vre.vrExperimentRegistration(mouseName, dateString, session, oasis=False, imaging=True)
vrReg.doPreprocessing()
vrReg.saveParams()
print(vrReg.sessionPath())
del vrReg

In [3]:
# Load registered vrExperiment
vrexp = vre.vrExperiment(mouseName, dateString, session)

In [13]:
%%time
speedThreshold = 5
omap, smap, lickmap, distvec = vrf.getBehaviorMaps(vrexp,speedThreshold=speedThreshold) # produce smoothed occupancy and speed maps, along with the distance bins used to create them
frameTrialIdx, framePosition, frameSpeed = vrexp.getFrameBehavior() # convert behavioral data into timeframe of spiking data
spkmap = vrf.getSpikeMap(vrexp, frameTrialIdx, framePosition, frameSpeed, distvec, omap, speedThreshold=speedThreshold, standardizeSpks=False, doSmoothing=0) # produce the spkmap 
idxFullTrials = np.all(~np.isnan(omap),axis=1) # find out which trials the mouse explored the whole environment
relmse, relcor = vrf.measureReliability(spkmap[:,idxFullTrials], numcv=3) # measure reliability of spiking
spInfo = vrf.measureSpatialInformation(omap[idxFullTrials], spkmap[:,idxFullTrials]) # measure spatial information of spiking

CPU times: total: 3.58 s
Wall time: 512 ms


In [147]:
# Code block for opening redSelectionGUI for a vrExperiment
vrexp = vre.vrExperiment(mouseName, dateString, session)
redCell = vre.redCellProcessing(vrexp)
redSelection = rgui.redSelectionGUI(redCell)

In [30]:
# fm.getCopyString('ATL027')
fm.s2pTargets('ATL028','2023-08-08','701')

\\zaru.cortexlab.net\Subjects\ATL028\2023-08-08\701
C:\Users\andrew\Documents\localData\ATL028\2023-08-08\701


In [35]:
# Database Ideas: 
# ---------------------
# GUI: db manager
# - show full database -
# - show selected entries (like for example entries with imaging but no suite2p) -
#                --> It would be sweet to make this work where there are some defaults, but then there is also a "filter command" option which you type and goes as the input to the filtering
# - click on entry and do things:
#                --> open file explorer to that session
#                --> do suite2p
#                --> do red cell management
# - update table data? 
# ---------------------
# Operational Commands: 
# - Automatically do suite2p 
# - ??? 

In [1]:
vrdb = dbm.vrDatabase()

In [36]:
df = vrdb.showTable()

PandasGUI INFO — pandasgui.gui — Opening PandasGUI


In [48]:
vrdb.checkS2P(withDatabaseUpdate=True)

Database said suite2p didn't run, but it already did: ATL028/2023-08-01/701
An exception occurred while trying to connect to vrDatabase!
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\andrew\anaconda3\envs\vrAnalysis\lib\site-packages\IPython\core\interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\andrew\AppData\Local\Temp\ipykernel_22768\2770492262.py", line 1, in <module>
    vrdb.checkS2P(withDatabaseUpdate=True)
  File "C:\Users\andrew\Documents\GitHub\vrAnalysis\dbManagement.py", line 108, in checkS2P
    cursor.execute(updateStatement)
pyodbc.Error: ('HY000', '[HY000] [Microsoft][ODBC Microsoft Access Driver] Operation must use an updateable query. (-3035) (SQLExecDirectW)')

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\andrew\anaconda3\envs\vrAnalysis\lib\site-packages\IPython\core\interactiveshell.py", line 2102, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "C:\Users\andrew\anaconda3\envs\vrAnalysis\lib\site-packages\IPython\core\ultratb.p

In [46]:
vrdb.printRequiresS2P()

Database indicates that suite2p has not been run: ATL012/2023-01-26/701
Database indicates that suite2p has not been run: ATL012/2023-01-26/702
Database indicates that suite2p has not been run: ATL012/2023-01-30/703
Database indicates that suite2p has not been run: ATL028/2023-08-01/701


In [38]:
len(vrdb.getTable())

92

In [39]:
# Before registering everything, do this: 
print("Convert mpciROIs.planeIndex to mpciROIs.stackPosition, with [X,Y,Z(planeIndex)] coordinate for each ROI!")

Convert mpciROIs.planeIndex to mpciROIs.stackPosition, with [X,Y,Z(planeIndex)] coordinate for each ROI!
